# List all files in  the wiki folder

We can create a list with the names of all files in the wiki folder using the `os.listdir()` function.

In [2]:
import os

file_names = os.listdir("../../Data/wiki")
len(file_names)

1004

In [5]:
'''
math: Used for mathematical operations.
functools: Provides higher-order functions that act on or return other functions.
Pool from multiprocessing: Manages a pool of worker processes.
'''
import math
import functools
from multiprocessing import Pool

# Read the first file

Let's read the first file and print its contents. We need to join the name of the file with the `wiki` folder. We can do this using the `os.path.join()` function.

In [ ]:
with open(os.path.join("../../Data/wiki", file_names[0])) as f:
    print(f.read())

# Adding the MapReduce function to this project

We start by adding the MapReduce function so that we can use throughout the project.

In [3]:
'''
Purpose: Divides the data into smaller chunks for parallel processing.make_chunks(data, num_chunks) Splits the input data into approximately equal-sized chunks.

Parameters:
    data: The data to be split.
    num_chunks: The number of chunks to split the data into.
    Returns: A list of data chunks.
Implementation:
    chunk_size: Calculates the size of each chunk using ceiling division.
    Uses list comprehension to create chunks of data.
'''
def make_chunks(data, num_chunks):
    chunk_size = math.ceil(len(data) / num_chunks)
    return [data[i:i+chunk_size] for i in range(0, len(data), chunk_size)]

In [6]:
'''
Purpose: Applies the map-reduce pattern to the data using multiple processes. map_reduce(data, num_processes, mapper, reducer): Implements the map-reduce pattern using the provided mapper and reducer functions.

Parameters:
    data: The data to be processed.
    num_processes: The number of processes to use for parallel processing.
    mapper: A function to apply to each chunk of data.
    reducer: A function to combine the results from the mapper functions.
Returns: The final reduced result.
Implementation:
    Splits the data into chunks using make_chunks.
    Creates a pool of worker processes.
    Applies the mapper function to each chunk in parallel using pool.map.
    Closes the pool and waits for all processes to finish.
    Combines the results from all chunks using the reducer function and functools.reduce.
'''
def map_reduce(data, num_processes, mapper, reducer):
    chunks = make_chunks(data, num_processes)
    pool = Pool(num_processes)
    chunk_results = pool.map(mapper, chunks)
    pool.close()
    pool.join()
    return functools.reduce(reducer, chunk_results)

# Counting the total number of lines on all files

It was not required but can use MapReduce to count the total number of lines on all files in the wiki folder! If you did in some other way, that is fine as well.

In [4]:
def map_line_count(file_names):
    total = 0
    for fn in file_names:
        with open(os.path.join("wiki", fn)) as f:
            total += len(f.readlines())
    return total
    
def reduce_line_count(count1, count2):
    return count1 + count2

target = "data"
map_reduce(file_names, 8, map_line_count, reduce_line_count)

499797

# Grep string function

We defined a `mapreduce_grep_string()` function that takes two arguments as input:

1. A path to a folder. In the case of this guided project we will only use it on the `wiki` folder but having this argument makes the function easier to reuse.

2. The string that we want to find.

The mapper function receives a chunk of filenames and calculates all occurrences of the target string on them. If a file contains no occurrences, we chose to not include an entry for that file in the result dictionary.

The reducer function uses the `dict.update()` method to merge the result dictionaries.

Note that the `target` variable will be defined outside and will be the string we are looking for.

In [5]:
# The target variable is defined outside and contains the string 
def map_grep(file_names):
    results = {}
    for fn in file_names:
        with open(fn) as f:
            lines = [line for line in f.readlines()]
        for line_index, line in enumerate(lines):
            if target in line:
                if fn not in results:
                    results[fn] = []
                results[fn].append(line_index)
    return results

def reduce_grep(lines1, lines2):
    lines1.update(lines2)
    return lines1

def mapreduce_grep(path, num_processes):
    file_names = [os.path.join(path, fn) for fn in os.listdir(path)]
    return map_reduce(file_names, num_processes,  map_grep, reduce_grep)

# Finding the occurences of "data"

In [6]:
target = "data"
data_occurrences = mapreduce_grep("wiki", 8)

# Allow for case insensitive matches

We can allow case insensitive matches by converting both the target and the file contents to lowercase before we match.

In [7]:
def map_grep_insensitive(file_names):
    results = {}
    for fn in file_names:
        with open(fn) as f:
            lines = [line.lower() for line in f.readlines()]
        for line_index, line in enumerate(lines):
            if target.lower() in line:
                if fn not in results:
                    results[fn] = []
                results[fn].append(line_index)
    return results

def mapreduce_grep_insensitive(path, num_processes):
    file_names = [os.path.join(path, fn) for fn in os.listdir(path)]
    return map_reduce(file_names, num_processes,  map_grep_insensitive, reduce_grep)

target = "data"
new_data_occurrences = mapreduce_grep_insensitive("wiki", 8)

# Checking that we find more matches

We already stored the results into variables `data_occurrences` and `new_data_occurrences`.  To check that we find more matches with the second version of the algorithm, we can loop over the file names and print the length difference between the results.

In [8]:
for fn in new_data_occurrences:
    if fn not in data_occurrences:
        print("Found {} new matches on file {}".format(len(new_data_occurrences[fn]), fn))
    elif len(new_data_occurrences[fn]) > len(data_occurrences[fn]):
        print("Found {} new matches on file {}".format(len(new_data_occurrences[fn]) - len(data_occurrences[fn]), fn))

Found 6 new matches on file wiki/Dragnet_(franchise).html
Found 1 new matches on file wiki/Jazz_in_Turkey.html
Found 2 new matches on file wiki/Kate_Harwood.html
Found 1 new matches on file wiki/Rally_for_Democracy_and_Progress_(Benin).html
Found 1 new matches on file wiki/Morning_Glory_(2010_film).html
Found 2 new matches on file wiki/Jules_Verne_ATV.html
Found 1 new matches on file wiki/Claudia_Neidig.html
Found 2 new matches on file wiki/Gordon_Bau.html
Found 1 new matches on file wiki/Colchester_Village_Historic_District.html
Found 1 new matches on file wiki/Sahanpur.html
Found 1 new matches on file wiki/Harry_Hill_Bandholtz.html
Found 1 new matches on file wiki/Morgana_King.html
Found 1 new matches on file wiki/Nuno_Leal_Maia.html
Found 1 new matches on file wiki/Alex_Kurtzman.html
Found 1 new matches on file wiki/Camp_Nelson_Confederate_Cemetery.html
Found 1 new matches on file wiki/Dewoitine_D.21.html
Found 1 new matches on file wiki/WLSR.html
Found 7 new matches on file wiki/Li

# Finding match indexes on lines

We need to solve a subproblem before we implement this one: Given a string and a target, find all occurrences of the target within that string.

In [9]:
def find_match_indexes(line, target):
    results = []
    i = line.find(target, 0)
    while i != -1:
        results.append(i)
        i = line.find(target, i + 1)
    return results

# Test implementation
s = "Data science is related to data mining, machine learning and big data.".lower()
print(find_match_indexes(s, "data"))

[0, 27, 65]


# Finding all match locations

We can use any of the above functions to find all match locations. We will use the third one.

After finding all indexes in one line, we need to create pairs by adding the line index.

In [10]:
def map_grep_match_indexes(file_names):
    results = {}
    for fn in file_names:
        with open(fn) as f:
            lines = [line.lower() for line in f.readlines()]
        for line_index, line in enumerate(lines):
            match_indexes = find_match_indexes(line, target.lower())
            if fn not in results:
                results[fn] = []
            results[fn] += [(line_index, match_index) for match_index in match_indexes]
    return results

def mapreduce_grep_match_indexes(path, num_processes):
    file_names = [os.path.join(path, fn) for fn in os.listdir(path)]
    return map_reduce(file_names, num_processes,  map_grep_match_indexes, reduce_grep)

target = "science"
occurrences = mapreduce_grep_match_indexes("wiki", 8)

# Displaying the results

Let's display the results. We will create a CSV file listing all occurrences. We will also show the text around each occurrence.

In [11]:
import csv

# How many character to show before and after the match
context_delta = 30

with open("results.csv", "w") as f:
    writer = csv.writer(f)
    rows = [["File", "Line", "Index", "Context"]]
    for fn in occurrences:
        with open(fn) as f:
            lines = [line.strip() for line in f.readlines()]
        for line, index in occurrences[fn]:
            start = max(index - context_delta, 0)
            end   = index + len(target) + context_delta
            rows.append([fn, line, index, lines[line][start:end]])
    writer.writerows(rows)

In [12]:
import pandas
df = pandas.read_csv("results.csv")
df.head(10)

,File,Line,Index,Context
0,wiki/Rally_for_Democracy_and_Progress_(Benin)....,155,40,"f=""/wiki/Outline_of_political_science#Politics..."
1,wiki/Rally_for_Democracy_and_Progress_(Benin)....,155,96,""" title=""Outline of political science"">Other c..."
2,wiki/Jules_Verne_ATV.html,208,507,"century French <a href=""/wiki/Science-fiction""..."
3,wiki/Jules_Verne_ATV.html,208,551,"n"" class=""mw-redirect"" title=""Science-fiction""..."
4,wiki/Jules_Verne_ATV.html,208,568,"rect"" title=""Science-fiction"">science-fiction<..."
5,wiki/Jules_Verne_ATV.html,427,231,"text"" href=""http://www.futura-sciences.com/fr/..."
6,wiki/Jules_Verne_ATV.html,427,427,"nnés""</a> (in French). Futura Sciences<span cl..."
7,wiki/Jules_Verne_ATV.html,427,831,ft_id=http%3A%2F%2Fwww.futura-sciences.com%2Ff...
8,wiki/Jules_Verne_ATV.html,427,971,15986-1%2F&amp;rft.pub=Futura+Sciences&amp;rft...
9,wiki/Jules_Verne_ATV.html,941,60,ogramme_for_Life_and_Physical_Sciences_in_Spac...
